![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_snowflake.ipynb)

# Import ONNX snowflake models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for snowflake from HuggingFace and they have to be in `Fill Mask` category. Meaning, you cannot use snowflake models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.52.3 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [Snowflake/snowflake-arctic-embed-m](https://huggingface.co/Snowflake/snowflake-arctic-embed-m) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the snowflake model, we also need to save the Tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "silvainrichou/snowflake-arctic-embed-m"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/219M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

('onnx_models/silvainrichou/snowflake-arctic-embed-m/tokenizer_config.json',
 'onnx_models/silvainrichou/snowflake-arctic-embed-m/special_tokens_map.json',
 'onnx_models/silvainrichou/snowflake-arctic-embed-m/vocab.txt',
 'onnx_models/silvainrichou/snowflake-arctic-embed-m/added_tokens.json',
 'onnx_models/silvainrichou/snowflake-arctic-embed-m/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [9]:
!ls -l {EXPORT_PATH}

total 426572
-rw-r--r-- 1 root root       590 Jun 16 21:11 config.json
-rw-r--r-- 1 root root 435844616 Jun 16 21:12 model.onnx
-rw-r--r-- 1 root root       695 Jun 16 21:12 special_tokens_map.json
-rw-r--r-- 1 root root      1463 Jun 16 21:12 tokenizer_config.json
-rw-r--r-- 1 root root    711396 Jun 16 21:12 tokenizer.json
-rw-r--r-- 1 root root    231508 Jun 16 21:12 vocab.txt


- We need to move `vocab.txt` from the tokenizer into an `assets` folder, which is where Spark NLP expects to find it.


In [11]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

In [12]:
!ls -l {EXPORT_PATH}

total 426348
drwxr-xr-x 2 root root      4096 Jun 16 21:17 assets
-rw-r--r-- 1 root root       590 Jun 16 21:11 config.json
-rw-r--r-- 1 root root 435844616 Jun 16 21:12 model.onnx
-rw-r--r-- 1 root root       695 Jun 16 21:12 special_tokens_map.json
-rw-r--r-- 1 root root      1463 Jun 16 21:12 tokenizer_config.json
-rw-r--r-- 1 root root    711396 Jun 16 21:12 tokenizer.json


Voila! We have our `vocab.txt` inside assets directory

## Import and Save snowflake in Spark NLP

- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 31.4 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `SnowFlakeEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `SnowFlakeEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [13]:
from sparknlp.annotator import SnowFlakeEmbeddings

snowflake = SnowFlakeEmbeddings.loadSavedModel(EXPORT_PATH, spark)\
    .setInputCols(["document"])\
    .setOutputCol("snowflake")\
    .setCaseSensitive(True)\
    .setDimension(768)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [14]:
snowflake.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [15]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX snowflake model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [16]:
! ls -l {MODEL_NAME}_spark_nlp

total 425708
drwxr-xr-x 3 root root      4096 Jun 16 21:18 fields
drwxr-xr-x 2 root root      4096 Jun 16 21:18 metadata
-rw-r--r-- 1 root root 435911255 Jun 16 21:18 SnowFlake_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny snowflake model 😊

In [17]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import SnowFlakeEmbeddings
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

snowflake_embeddings = SnowFlakeEmbeddings.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document"]) \
    .setOutputCol("snowflake")

pipeline = Pipeline(stages=[
    document_assembler,
    snowflake_embeddings
])

data = spark.createDataFrame([
    ["William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."]
], ["text"])

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("explode(snowflake.embeddings) as embedding").show()

+--------------------+
|           embedding|
+--------------------+
|[-0.60493106, 0.5...|
+--------------------+



That's it! You can now go wild and use hundreds of snowflake models from HuggingFace 🤗 in Spark NLP 🚀
